In [ ]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import albumentations as A  
import matplotlib.pyplot as plt

# 1. SETTING PATHS
DATA_DIR = "images"                # folder with images
ANNOTATION_FILE = "faces.csv"  # bounding box annotations

# 2. LOADING ANNOTATIONS
df_annotations = pd.read_csv(ANNOTATION_FILE)

print("Total annotations:", len(df_annotations))

# 3. REMOVING DUPLICATES OR IRRELEVANT ENTRIES
df_annotations.drop_duplicates(inplace=True)

# 4. CLEANING ANY INCORRECT ANNOTATIONS
# Removing negative or zero-size bounding boxes
df_annotations = df_annotations[
    (df_annotations['x1'] > df_annotations['x0']) &
    (df_annotations['y1'] > df_annotations['y0'])
]

# 5. RESIZING IMAGES 
# Resizing all images to 224x224
RESIZE_WIDTH = 224
RESIZE_HEIGHT = 224

processed_images_dir = "data/images_processed"
os.makedirs(processed_images_dir, exist_ok=True)

new_rows = []  # to store updated bounding boxes after resizing

for idx, row in tqdm(df_annotations.iterrows(), total=len(df_annotations)):
    img_path = os.path.join(DATA_DIR, row['image_name'])
    if not os.path.exists(img_path):
        continue
    img = cv2.imread(img_path)
    original_height, original_width = row['height'], row['width']

    # Resizing
    img_resized = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT))
    
    # Adjusting bounding boxes proportionally
    x_scale = RESIZE_WIDTH / original_width
    y_scale = RESIZE_HEIGHT / original_height

    x_min_new = int(row['x0'] * x_scale)
    y_min_new = int(row['y0'] * y_scale)
    x_max_new = int(row['x1'] * x_scale)
    y_max_new = int(row['y1'] * y_scale)

    # Saving resized image
    new_image_name = f"resized_{row['image_name']}"
    new_image_path = os.path.join(processed_images_dir, new_image_name)
    cv2.imwrite(new_image_path, img_resized)

    new_rows.append([
        new_image_name, RESIZE_WIDTH, RESIZE_HEIGHT, x_min_new, y_min_new, x_max_new, y_max_new
    ])

# Creating DataFrame for resized annotations
df_annotations_resized = pd.DataFrame(new_rows,
    columns=['image_name', 'width', 'height', 'x0', 'y0', 'x1', 'y1'])

df_annotations_resized.to_csv("data/resized_annotations.csv", index=False)

# 6. DATA AUGMENTATION
augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Rotate(limit=15, p=0.3)
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

aug_rows = []
for idx, row in tqdm(df_annotations_resized.iterrows(), total=len(df_annotations_resized)):
    image_path = os.path.join(processed_images_dir, row['image_name'])
    image = cv2.imread(image_path)
    bboxes = [[row['x0'], row['y0'], row['x1'], row['y1']]]
    class_labels = ["face"]  # Assuming all images are faces
    
    transformed = augmentation(image=image, bboxes=bboxes, class_labels=class_labels)
    aug_img = transformed['image']
    aug_bboxes = transformed['bboxes']
    aug_labels = transformed['class_labels']
    
    # Save augmented image
    aug_image_name = f"aug_{idx}_{row['image_name']}"
    aug_image_path = os.path.join(processed_images_dir, aug_image_name)
    cv2.imwrite(aug_image_path, aug_img)
    
    for box in aug_bboxes:
        x_min_aug, y_min_aug, x_max_aug, y_max_aug = map(int, box)
        aug_rows.append([aug_image_name, RESIZE_WIDTH, RESIZE_HEIGHT, x_min_aug, y_min_aug, x_max_aug, y_max_aug])

df_annotations_aug = pd.DataFrame(aug_rows, 
                                  columns=['image_name', 'width', 'height', 'x0', 'y0', 'x1', 'y1'])

# Combining original + augmented
df_final = pd.concat([df_annotations_resized, df_annotations_aug], ignore_index=True)
df_final.to_csv("data/final_annotations.csv", index=False)

print("Preprocessing done. Final annotation count:", len(df_final))

Total annotations: 3350


100%|██████████| 3350/3350 [00:12<00:00, 272.46it/s]

Preprocessing done. Final annotation count: 6700


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import cv2
import os

# Loading final annotations
df_final = pd.read_csv("data/final_annotations.csv")

# 1. Image Count
unique_images = df_final['image_name'].nunique()
print("Total unique images:", unique_images)

# 2. Face Count
face_count = len(df_final)
print("Total faces in the dataset:", face_count)

# 3. Faces per image distribution
faces_per_image = df_final.groupby('image_name').size().reset_index(name='count')
fig_faces = px.histogram(faces_per_image, x='count', nbins=10,
                         title="Distribution of number of faces per image")
fig_faces.show()

# 4. Bounding Box Accuracy Check
# Checking if bounding boxes are in a valid range 
invalid_bboxes = df_final[
    (df_final['x0'] < 0) | (df_final['y0'] < 0) |
    (df_final['x1'] <= df_final['x0']) |
    (df_final['y1'] <= df_final['y0'])
]
print("Invalid bounding boxes count:", len(invalid_bboxes))

# 5. Resolution distribution (since all are resized to 224x224, it might be uniform)

# 6. Quick visualization of bounding box sizes
df_final['bbox_width'] = df_final['x1'] - df_final['x0']
df_final['bbox_height'] = df_final['y1'] - df_final['y0']

fig_bbox_size = px.scatter(df_final, x='bbox_width', y='bbox_height',
                           title="Scatter Plot of Bounding Box Width vs Height")
fig_bbox_size.show()

print("EDA complete.")

Total unique images: 5554
Total faces in the dataset: 6700


Invalid bounding boxes count: 0


EDA complete.


In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Loadig the final CSV which includes original + augmented
df = pd.read_csv("data/final_annotations.csv")

# Splitting dataset into train / val / test
df_train, df_temp = train_test_split(df, test_size=0.2, random_state=42)
df_val, df_test = train_test_split(df_temp, test_size=0.5, random_state=42)

print("Training set size:", len(df_train))
print("Validation set size:", len(df_val))
print("Test set size:", len(df_test))

# Creating YOLO directory structure
os.makedirs("data/images/train", exist_ok=True)
os.makedirs("data/images/val", exist_ok=True)
os.makedirs("data/images/test", exist_ok=True)
os.makedirs("data/labels/train", exist_ok=True)
os.makedirs("data/labels/val", exist_ok=True)
os.makedirs("data/labels/test", exist_ok=True)

def convert_and_copy(df_split, split_name):
    for idx, row in df_split.iterrows():
        img_name = row['image_name']
        width = row['width']
        height = row['height']
        x0 = row['x0']
        y0 = row['y0']
        x1 = row['x1']
        y1 = row['y1']

        # Source is the processed images directory
        src_img_path = os.path.join("data/images_processed", img_name)
        if not os.path.exists(src_img_path):
            continue
        
        # Destination path for this split
        dst_img_path = os.path.join(f"data/images/{split_name}", img_name)
        
        if not os.path.exists(dst_img_path):
            shutil.copy(src_img_path, dst_img_path)

        # Creating YOLO label file name
        label_file_name = os.path.splitext(img_name)[0] + ".txt"
        label_path = os.path.join(f"data/labels/{split_name}", label_file_name)

        # Converting bounding box to normalized YOLO format:
        #   class_id x_center y_center w h
        #   all normalized to [0,1]
        x_center = ((x0 + x1) / 2) / width
        y_center = ((y0 + y1) / 2) / height
        w_norm = (x1 - x0) / width
        h_norm = (y1 - y0) / height

        # Single-class case => class_id = 0 (face)
        yolo_line = f"0 {x_center} {y_center} {w_norm} {h_norm}\n"

        # Appending to label file in case multiple bboxes per image
        with open(label_path, "a") as f:
            f.write(yolo_line)

# Processing each split
convert_and_copy(df_train, "train")
convert_and_copy(df_val, "val")
convert_and_copy(df_test, "test")

print("YOLO dataset structure created under data/images and data/labels.")

Training set size: 5360
Validation set size: 670
Test set size: 670
YOLO dataset structure created under data/images and data/labels.


In [ ]:
import os

project_dir = r"C:\Users\suraa\OneDrive\Documents\Final Project"
train_dir = os.path.join(project_dir, "data", "images", "train")
val_dir   = os.path.join(project_dir, "data", "images", "val")
test_dir  = os.path.join(project_dir, "data", "images", "test")

# Replacing backslashes with forward slashes for compatibility
train_dir = train_dir.replace("\\", "/")
val_dir = val_dir.replace("\\", "/")
test_dir = test_dir.replace("\\", "/")

data_yaml_content = f"""# YOLOv8 data config
train: {train_dir}
val: {val_dir}
test: {test_dir}

# List of class names
names:
  0: face
"""

with open("data.yaml", "w") as f:
    f.write(data_yaml_content)

print("data.yaml created successfully.")

data.yaml created successfully.


In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
results = model.train(
    data='data.yaml',
    epochs=50,
    imgsz=224,
    batch=8,
    name='face_detection'
)

Ultralytics 8.3.78  Python-3.10.0 torch-2.6.0+cpu CPU (Intel Core(TM) i7-14700HX)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=face_detection5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

train: Scanning C:\Users\suraa\OneDrive\Documents\Final Project\data\labels\train.cache... 4533 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4533/4533 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\Users\suraa\OneDrive\Documents\Final Project\data\labels\val.cache... 643 images, 0 backgrounds, 0 corrupt: 100%|██████████| 643/643 [00:00<?, ?it/s]


Plotting labels to runs\detect\face_detection5\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs\detect\face_detection5
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      50/50         0G     0.9776     0.8047      1.173          7        224: 100%|██████████| 567/567 [01:33<00:00,  6.04it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:04<00:00,  8.33it/s]

                   all        643        670      0.751      0.607      0.798      0.605



50 epochs completed in 1.441 hours.
Optimizer stripped from runs\detect\face_detection5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\face_detection5\weights\best.pt, 6.2MB

Validating runs\detect\face_detection5\weights\best.pt...
Ultralytics 8.3.78  Python-3.10.0 torch-2.6.0+cpu CPU (Intel Core(TM) i7-14700HX)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:04<00:00,  9.44it/s]


                   all        643        670       0.75      0.607      0.798      0.605
Speed: 0.1ms preprocess, 5.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\face_detection5


In [ ]:
from ultralytics import YOLO
model = YOLO("C:/Users/suraa/OneDrive\Documents/Final Project/runs/detect/face_detection5/weights/best.pt")  
metrics = model.val(data="data.yaml")
print(metrics)

Ultralytics 8.3.78  Python-3.10.0 torch-2.6.0+cpu CPU (Intel Core(TM) i7-14700HX)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\suraa\OneDrive\Documents\Final Project\data\labels\val.cache... 643 images, 0 backgrounds, 0 corrupt: 100%|██████████| 643/643 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 41/41 [00:04<00:00,  8.90it/s]


                   all        643        670       0.75      0.607      0.798      0.605
Speed: 0.1ms preprocess, 5.6ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs\detect\val3
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000020563D1BF70>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.0

In [6]:
import json
mp, mr, map50, map_ = metrics.box.mean_results()

metrics_dict = {
    "mAP@0.5": map50,
    "mAP@0.5:0.95": map_,
    "Precision": mp,
    "Recall": mr
}

with open("metrics.json", "w") as f:
    json.dump(metrics_dict, f, indent=4)
